# Flower Federated Learning Tutorial

The tutorial demonstrates how to use the Flower FL framework for execution of federated learning tasks. The tutorial is based on official Pandas example of Flower framework.

## Initialize the project

In [ ]:
import digitalhub as dh

project = dh.get_or_create_project("test-flower")

# Create and Start Server part: Superlink

The following defines the Superlink function, build the corresponding image, and activates the server container with the dependencies.  
Note that the server is started in `insecure` mode, meaning that no TLS verification is performed. While this is ok for the purpose of this tutorial, should not be used in a production federation.

In secure mode the server will be equipped with a custom platform-level TLS certificate that should also be used by client SuperNode nodes.

In [ ]:
server_function = project.new_function(
    name="my-flower-server",
    kind="flower-server",
    requirements=["pandas==2.2.3", "flwr-datasets[vision]==0.5.0"]
)
# Build the server
server_function.run(action="build", wait=True)

# Deploy the server
run = server_function.run(action="deploy", insecure=True)

# Create and Start Client part: Supernode

The following defines the Supernode function, build the corresponding image, and activates 3 client nodes container with the dependencies and the node-specific configuration.  

Note that the server is started in `insecure` mode as no root certificate is provided. To enable secure mode, it is necessary to specify the `root_certificates` attribute to each run containing the body of the public root certificate.

Also the node authentication is not enabled in this scenario. The node authentication allows for controlling which nodes are allowed to communicate with the server. To achieve this, it is necessary

- at each node define a public-private key pair and store the values in project secrets.
- pass the secret names as `public_key_secret` and `private_key_secret` parameters to the supernode spec
- ensure the public key is included in `auth_public_keys` field of the server Superlink node

Each client is started with the own set of parameters (`node_config`) and a reference to the `superlink` pointing to the superlink address (port 9092 by default).

In [ ]:
server_url = run.refresh().status.service['url'].split(':')[0] + ':9092' 

client_function = project.new_function(
    name="my-flower-client",
    kind="flower-client",
    requirements=["pandas==2.2.3", "flwr-datasets[vision]==0.5.0"]
)

client_function.run(action="build", wait=True)

# Deploy client 1
run = client_function.run(action="deploy", superlink=server_url, node_config={
        "partition-id": 0,
        "num-partitions": 3,
        "local-epochs": 2
})

# Deploy client 2
run = client_function.run(action="deploy", superlink=server_url, node_config={
        "partition-id": 1,
        "num-partitions": 3,
        "local-epochs": 2
})

# Deploy client 3
run = client_function.run(action="deploy", superlink=server_url, node_config={
        "partition-id": 2,
        "num-partitions": 3,
        "local-epochs": 2
})

# Create and Start the training execution

To perform the actual training procedure, we define a new flower app function with the application code for client and server.
Specifically, it is necessary to provide either the reference to a complete Git project or, as in this case, the reference to the client and server source code files:

- `client_src` provides reference to the  client source code, while `client_app` defines the reference in the code to the ClientApp instance.
- `server_src` provides reference to the  server source code, while `server_app` defines the reference in the code to the ServerApp instance.

In [ ]:
app_function = project.new_function(
    name="my-flower-app",
    kind="flower-app",
    client_src="src/client.py",
    server_src="src/server.py",
    client_app="app",
    server_app="app"

)

The execution of the application run is a single job that bundles the application and interacts with the superlink to deploy, activate, and coordinate the federated learning execution. 

Each execution requires the reference to the corresponding execution API of the Superlink (port 9093) and optionally a set of hyperparameters.

Note that the server is started in `insecure` mode as no root certificate is provided. To enable secure mode, it is necessary to specify the `root_certificates` attribute to each run containing the body of the public root certificate.

In [12]:
server_url = server_url.split(':')[0] + ':9093' 

app_run = app_function.run("train", superlink=server_url, parameters={
    "num-config-rounds":3,
    "fraction-sample": 1.0
})

The status and log of the execution may be obtained from the corresponding app run, e.g., through the Core Web UI.